# Overview
This exploration is part of the AskGrowBuddy project that uses notes from Obsidian to provide growing knowledge.  The notebook explores having an LLM generate questions from a bunch of Obsidian notes.  To do this, the Langchain Obsidian document loader is used to load documents from a directory within an Obsidian vault. The Langchain Obsidian loader is used because it understands how to convert frontmatter to metadata. Obsidian notes typically hold metadata that is helpful in RAG.  I am not exploring metadata here but it is something to look into.

# Load Obsidian Notes
Load the Obsidian vault directory into Langchain documents using Langchain's obsidian loader.

# Set up nest_asyncio

In [13]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from src.ingest_service import IngestService

In [15]:
ingest_service = IngestService()
docs = ingest_service.load_docs(r"G:\My Drive\Audios_To_Knowledge\knowledge\AskGrowBuddy\AskGrowBuddy\Knowledge\Question_Answer")

# Check Content
Random check to see if the content was loaded.

In [16]:
def search_content(docs, search_text):
    for i, doc in enumerate(docs):
        if search_text.lower() in doc.page_content.lower():
            print(f"Found in document {i}:")
            print("Metadata:")
            for key, value in doc.metadata.items():
                print(f"  {key}: {value}")
            print("\nContent preview:")
            print(doc.page_content[:200] + "...")
            print("\n---\n")

# Example usage
search_text = "A new book from Steve Solomon is reason "
search_content(docs, search_text)

In [17]:
len(docs[0].page_content)


1111

# Convert Langchain docs to llamaindex docs

In [18]:
# Convert langchain Document to llamaindex document
from llama_index.core import Document as LlamaDocument

def convert_to_llama_documents(langchain_docs):
    llama_docs = []
    for lc_doc in langchain_docs:
        llama_doc = LlamaDocument(
            text=lc_doc.page_content,
            metadata=lc_doc.metadata
        )
        llama_docs.append(llama_doc)
    return llama_docs

# Convert the documents
llama_index_docs = convert_to_llama_documents(docs)

# Print the first document to verify
if llama_index_docs:
    print(f"Number of docs: {len(llama_index_docs)}")
    print("First LlamaIndex Document:")
    print(f"Text preview: {llama_index_docs[0].text[:100]}...")
    print(f"Metadata: {llama_index_docs[0].metadata}")
else:
    print("No documents were converted.")

Number of docs: 12
First LlamaIndex Document:
Text preview: # Sulfur

Sulfur, in partnership with nitrogen, forms key pieces in several essential amino acids an...
Metadata: {'source': 'Sulfur.md', 'path': 'G:\\My Drive\\Audios_To_Knowledge\\knowledge\\AskGrowBuddy\\AskGrowBuddy\\Knowledge\\Question_Answer\\soil test comments\\Sulfur.md', 'created': 1726950458.11, 'last_modified': 1725585368.506, 'last_accessed': 1726950458.11}


# Set up Ollama

In [19]:
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

# Set embedding model
ollama_embedding = OllamaEmbedding(
    model_name="all-minilm",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

# Configure Ollama LLM
llm = Ollama(model="llama3.1", temperature=0, request_timeout=1000.0)
#
# Configure Settings singleton
Settings.embed_model = ollama_embedding
Settings.llm = llm

# Set up Token Tracking
I want to know how many tokens are being used with the LLM in case I want to use a paid service.

In [20]:
import tiktoken
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler
from llama_index.core import Settings

# Create a TokenCountingHandler instance
token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)
Settings.callback_manager = CallbackManager([token_counter])


I am getting PoolTimeout.  This means the connection pool used by the client to make requests to Ollama has reached its maximum capacity and no additional connections are available within the specified timeout period.
- Adding time to `llm = Ollama(model="llama3.1", temperature=0, request_timeout=1000.0)`. The default for request_timeout is 30.0. Now it is still running 15 minutes later.  We'll see.
- The request_timeout had it running a lot longer. But ultimately came to the same failure point.  I am thinking it is the size of the number of documents. Maybe it is made to best ask questions over one document.
- now trying adding num_questions_per_chunk=1.

In [21]:
from llama_index.core.llama_dataset.generator import  RagDatasetGenerator
from llama_index.core.prompts import Prompt
 # generate questions
# %env OLLAMA_KEEP_ALIVE=3600
data_generator = RagDatasetGenerator.from_documents(
    llama_index_docs,
    num_questions_per_chunk=1,
    show_progress=True,
    text_question_template=Prompt(
        "A sample from the LlamaIndex documentation is below.\n"
        "---------------------\n"
        "{context_str}\n"
        "---------------------\n"
        "Using the documentation sample, carefully follow the instructions below:\n"
        "{query_str}"
    ),
    question_gen_query=(
        "You are an evaluator for a search pipeline. Your task is to write a single question "
        "using the provided documentation sample above to test the search pipeline. The question should "
        "reference specific names, functions, and terms. Restrict the question to the "
        "context information provided.\n"
        "Question: "
    )
)
generated_questions = data_generator.generate_questions_from_nodes()


100%|██████████| 31/31 [02:31<00:00,  4.90s/it]


In [22]:
len(data_generator.nodes)

31

In [23]:
rag_dataset = data_generator.generate_questions_from_nodes()


100%|██████████| 31/31 [02:37<00:00,  5.07s/it]


In [24]:
rag_dataset.to_pandas()

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,Here's a question that tests the search pipeli...,"[# Sulfur\n\nSulfur, in partnership with nitro...",,None,ai (llama3.1)
1,Here's a question that can be used to test the...,[#soil_test #cation_ratio #M3 #Mehlic-3 #satu...,,None,ai (llama3.1)
2,Here's a question that can be used to test the...,[```mermaid\npie\n title Cation Ration\n ...,,None,ai (llama3.1)
3,Here's a question that can be used to test the...,[#soil_test #M3 #Mehlic-3 #SP #saturated_past...,,None,ai (llama3.1)
4,Here's a question that can be used to test the...,[#soil_test #M3 #Mehlic-3 #SP #saturated_past...,,None,ai (llama3.1)
5,Here's a question that can be used to test the...,[# my comments on 2021-11 Results\n\n[[SP_2023...,,None,ai (llama3.1)
6,Here's a question based on the provided docume...,"[This grow period, I liberally applied Mammoth...",,None,ai (llama3.1)
7,Here's a question based on the provided docume...,[### SP - Available Phosphorous\n**SP Target ...,,None,ai (llama3.1)
8,Here's a question that can be used to test the...,[Brandon stresses that while it is essential t...,,None,ai (llama3.1)
9,Here's a question that can be used to test the...,[## Magnesium\n\n![[M3 Magnesium lbs_acre.png|...,,None,ai (llama3.1)


In [25]:
print(
    "Embedding Tokens: ",
    token_counter.total_embedding_token_count,
    "\n",
    "LLM Prompt Tokens: ",
    token_counter.prompt_llm_token_count,
    "\n",
    "LLM Completion Tokens: ",
    token_counter.completion_llm_token_count,
    "\n",
    "Total LLM Token Count: ",
    token_counter.total_llm_token_count,
    "\n",
)

Embedding Tokens:  0 
 LLM Prompt Tokens:  54544 
 LLM Completion Tokens:  6176 
 Total LLM Token Count:  60720 

